In [1]:
import requests
import pandas as pd
import numpy as np
import gmaps
import json
import csv
from keys import gapikey
from matplotlib.cm import brg
from matplotlib.colors import to_hex
from area import area

gmaps.configure(api_key=gapikey)
with open('zcta/zcta.json') as f:
    geoj = json.load(f)
with open('resources/zip_targets.csv', newline='\n', encoding='utf-8-sig') as f:
    ziptgt = pd.read_csv(f)
with open('resources/dc_pop.csv', newline='\n', encoding='utf-8-sig') as f:
    dc_popdf = pd.read_csv(f)

In [2]:
dc_zippop = dc_popdf[['GEO.id2','HC01_EST_VC01']]
dc_zippop = dc_zippop.rename(columns={'GEO.id2': 'zip', 'HC01_EST_VC01': 'population'})
zippopdf = ziptgt.merge(dc_zippop, on='zip', how='outer').set_index('zip')
zippopdf = zippopdf.fillna(value=0)
zipdict = dict(zippopdf['population'])



In [3]:
poplist = []
geoj_scrubbed = {}
features = geoj['features']
feature_col = []
zip_areas = []
for feature in features:
    try:
        feature_geoid = int(feature['properties']['GEOID10'])
        if feature_geoid in zipdict.keys():
            feature_col.append(feature)
            poplist.append(zipdict[feature_geoid])
            zarea = area(feature['geometry'])/int(1E6)
            zip_areas.append(zarea)
            
    except ValueError:
            print(f'Value Error {b}')
                  
geoj_scrubbed = {'type': 'FeatureCollection', 'features': feature_col}

pop_density = [poplist[i]/ zip_areas[i] for i in range(len(poplist))]



In [4]:
def calculate_color(pop):
    min_pop = min(poplist)
    max_pop = max(poplist)
    pop_range = max_pop - min_pop
    normalized_pop = ((pop - min_pop)/pop_range)
    mpl_color = brg(normalized_pop)
    gmaps_color = to_hex(mpl_color, keep_alpha=False)
    
    return gmaps_color

colors = []

for pop in poplist:
    if int(pop) > 0:
        try:
            color = calculate_color(pop)
        except KeyError:
            color = (0, 0, 0, 0.3)
    else:
        color = (0, 0, 0, 0.3)
    colors.append(color)
zipdict

{20001: 41692.0,
 20002: 52867.0,
 20003: 26330.0,
 20004: 1610.0,
 20005: 12311.0,
 20006: 2780.0,
 20007: 23866.0,
 20008: 26034.0,
 20009: 46797.0,
 20010: 28736.0,
 20011: 54799.0,
 20012: 12565.0,
 20013: 0.0,
 20015: 12349.0,
 20016: 29230.0,
 20017: 16815.0,
 20018: 16617.0,
 20019: 47261.0,
 20020: 41842.0,
 20022: 0.0,
 20023: 0.0,
 20024: 11342.0,
 20026: 0.0,
 20027: 0.0,
 20029: 0.0,
 20030: 0.0,
 20032: 30806.0,
 20033: 0.0,
 20035: 0.0,
 20036: 5653.0,
 20037: 14362.0,
 20038: 0.0,
 20039: 0.0,
 20040: 0.0,
 20041: 0.0,
 20042: 0.0,
 20043: 0.0,
 20044: 0.0,
 20045: 0.0,
 20046: 0.0,
 20047: 0.0,
 20049: 0.0,
 20050: 0.0,
 20051: 0.0,
 20052: 686.0,
 20053: 0.0,
 20055: 0.0,
 20056: 0.0,
 20057: 3709.0,
 20058: 0.0,
 20059: 0.0,
 20060: 0.0,
 20061: 0.0,
 20062: 0.0,
 20063: 0.0,
 20064: 2213.0,
 20065: 0.0,
 20066: 0.0,
 20067: 0.0,
 20068: 0.0,
 20069: 0.0,
 20070: 0.0,
 20071: 0.0,
 20073: 0.0,
 20074: 0.0,
 20075: 0.0,
 20076: 0.0,
 20077: 0.0,
 20078: 0.0,
 20080: 0.

In [5]:
zcta_layer = gmaps.geojson_layer(
    geoj_scrubbed,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
fig = gmaps.figure()
fig.add_layer(zcta_layer)
fig

Figure(layout=FigureLayout(height='420px'))